<img src="https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/imagens/logo_nlportugues.png"   width="150" align="right">


# Lista 6 - LSTM & GRU


______________



O objetivo desta lista é fazer com que vocês treinem um modelo de análise de sentimentos utilizando GRU's e LSTM's. Essa lista é semelhante a lista 03 onde aprendemos a usar embeddings e onde você ja recebeu a arquitetura do seu modelo quase pronta. A diferença é que desta vez você ira construir sozinho sua rede e utilizará as camadas que acabamos de aprender: LSTM e GRU.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.13.0'

## Importando os dados como um dataframe

Para esta lista nós utilizaremos um recorte do dataset **B2W-Reviews01** que consiste em avaliações de mais de 130k compras online no site Americanas.com e [esta disponivel no github](https://github.com/b2wdigital/b2w-reviews01) sob a licensa CC BY-NC-SA 4.01.

In [ ]:
# !mkdir data

In [ ]:
# !curl https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2003/data/b2w-10k.csv --output 'data/b2w-10k.csv'

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/alan-barzilay/NLPortugues/master/Semana%2003/data/b2w-10k.csv'
r = requests.get(url)

with open('data/b2w-10k.csv', 'wb') as f:
    f.write(r.content)

In [26]:
# b2wCorpus = pd.read_csv("data/b2w-Reviews01.csv") # dataset completo
b2wCorpus = pd.read_csv("data/b2w-10k.csv") # dataset amostra
b2wCorpus.columns

Index(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'overall_rating', 'recommend_to_a_friend',
       'review_text', 'reviewer_birth_year', 'reviewer_gender',
       'reviewer_state', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')

In [4]:
df = b2wCorpus[['review_text', 'overall_rating']].copy()

# Entrega da Semana

In [34]:
def pipeline(path_file_input, path_file_output, prop_treino = 0.75, prop_teste=0.25):
    
    dados = pd.read_csv(path_file_input)
    dados_filtro = dados.loc[dados['overall_rating'].isin(list(range(6)))].copy()

    X = dados_filtro['review_text']
    y = dados_filtro['overall_rating']
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=prop_teste, random_state=42)
    
    dados_treino = pd.DataFrame({'review_text': X_treino, 'overall_rating': y_treino})
    dados_teste = pd.DataFrame({'review_text': X_teste, 'overall_rating': y_teste})
    
    dados_treino.to_csv(f'{path_file_output}/treino.csv', index=False)
    dados_teste.to_csv(f'{path_file_output}/teste.csv', index=False)
    
    # Aqui você pode adicionar a parte de codificação com Word2Vec ou outro método
    
    return dados_treino, dados_teste


In [35]:
df['overall_rating'].unique().tolist()

[4, 5, 1, 2, 3]

In [36]:
df = df[~df['review_text'].isna()]

In [37]:
df['review_text'].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
132368    False
132369    False
132370    False
132371    False
132372    False
Name: review_text, Length: 129098, dtype: bool

In [38]:
df.head()

,review_text,overall_rating
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4
4,"A entrega foi no prazo, as americanas estão de...",5


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import os

# Filtro de dados
def filtro(dados):
    return dados.loc[(dados['overall_rating'].isin(list(range(6)))) & (~dados['review_text'].isna())].copy()

# Partilha de dados
def partilha(path_file_input, path_file_output, prop_treino=0.75, prop_teste=0.25):
    dados = pd.read_csv(path_file_input)
    dados_filtrados = filtro(dados)

    X = dados_filtrados['review_text']
    y = dados_filtrados['overall_rating']

    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=prop_teste, random_state=42)

    dados_treino = pd.DataFrame({'review_text': X_treino, 'overall_rating': y_treino})
    dados_teste = pd.DataFrame({'review_text': X_teste, 'overall_rating': y_teste})

    os.makedirs(path_file_output, exist_ok=True)
    dados_treino.to_csv(f'{path_file_output}/treino.csv', index=False)
    dados_teste.to_csv(f'{path_file_output}/teste.csv', index=False)

    return dados_treino, dados_teste

# Codificação de palavras
def codifica(texto, d=100):
    sentences = [review.split() for review in texto]
    model_w2v = Word2Vec(sentences, vector_size=d, window=5, min_count=1, workers=4)
    return model_w2v

# Exemplo de uso
path_file_input = "data/b2w-10k.csv"
path_file_output = 'data/output'
prop_treino = 0.75
prop_teste = 0.25
d = 100

dados_treino, dados_teste = partilha(path_file_input, path_file_output, prop_treino, prop_teste)
model_w2v = codifica(dados_treino['review_text'], d)


In [40]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def truncar_texto(texto, max_length):
    return [sentenca[:max_length] for sentenca in texto]

def padding_texto(texto, max_length, token_pad):
    return pad_sequences(texto, maxlen=max_length, padding='post', truncating='post', value=token_pad)

def preparar_batches(X, y, batch_size, max_length, token_pad):
    X_pad = padding_texto(X, max_length, token_pad)
    num_batches = len(X_pad) // batch_size
    X_batches = np.array_split(X_pad[:num_batches*batch_size], num_batches)
    y_batches = np.array_split(y[:num_batches*batch_size], num_batches)
    return X_batches, y_batches

def preparar_vocabulario(texto, tamanho_vocabulario, token_desconhecido):
    palavras_frequentes = [palavra for lista_palavras in texto for palavra in lista_palavras]
    vocabulario = set(palavras_frequentes)
    vocabulario = {palavra: indice+1 for indice, palavra in enumerate(vocabulario)}
    vocabulario[token_desconhecido] = 0
    return vocabulario

def tokenizar_texto(texto, vocabulario, token_desconhecido):
    return [[vocabulario.get(palavra, 0) for palavra in sentenca] for sentenca in texto]


In [41]:
model_w2v

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, GRU, Dense

def treinar_modelo(X_treino, y_treino, X_teste, y_teste, tipo_modelo, bidirecional=False):
    modelo = Sequential()
    modelo.add(Embedding(input_dim=len(vocabulario), output_dim=100, input_length=max_length))
    if bidirecional:
        modelo.add(Bidirectional(tipo_modelo))
    else:
        modelo.add(tipo_modelo)
    modelo.add(Dense(6, activation='softmax'))
    modelo.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    modelo.fit(X_treino, y_treino, epochs=10, batch_size=64, validation_data=(X_teste, y_teste), verbose=0)
    return modelo

resultados = []
# Definindo o tamanho máximo da sequência e o token de padding
max_length = 100
token_pad = 0

# Tokenizar os textos
vocabulario = preparar_vocabulario(dados_treino['review_text'], 20000, '<UNK>')
X_treino = tokenizar_texto(dados_treino['review_text'], vocabulario, '<UNK>')
X_teste = tokenizar_texto(dados_teste['review_text'], vocabulario, '<UNK>')

y_treino = dados_treino['overall_rating'].values
y_teste = dados_teste['overall_rating'].values
# Truncar e fazer o padding dos textos
X_treino = truncar_texto(X_treino, max_length)
X_treino = padding_texto(X_treino, max_length, token_pad)
X_teste = truncar_texto(X_teste, max_length)
X_teste = padding_texto(X_teste, max_length, token_pad)

In [43]:
melhor_acuracia = 0
melhor_modelo = None
resultados = []

for tipo_modelo in ['LSTM', 'GRU']:
    for bidirecional in [False, True]:
        if tipo_modelo == 'LSTM':
            modelo = treinar_modelo(X_treino, y_treino, X_teste, y_teste, LSTM(units=128), bidirecional)
        elif tipo_modelo == 'GRU':
            modelo = treinar_modelo(X_treino, y_treino, X_teste, y_teste, GRU(units=128), bidirecional)
        acuracia = modelo.evaluate(X_teste, y_teste, verbose=0)[1]
        melhor = 'S' if acuracia == melhor_acuracia else 'N'
        resultados.append({'Método': f'{tipo_modelo}{" bi-direcional" if bidirecional else " uni-direcional"}',
                           'Acurácia': acuracia,
                           'Melhor (S/N)': melhor})

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

                Método  Acurácia Melhor (S/N)
0  LSTM uni-direcional    0.4032            N
1   LSTM bi-direcional    0.5000            N
2   GRU uni-direcional    0.4612            N
3    GRU bi-direcional    0.5072            N


In [44]:
df_resultados

,Método,Acurácia,Melhor (S/N)
0,LSTM uni-direcional,0.4032,N
1,LSTM bi-direcional,0.5000,N
2,GRU uni-direcional,0.4612,N
3,GRU bi-direcional,0.5072,N


In [31]:
df_resultados.loc[df_resultados['Acurácia'] == df_resultados['Acurácia'].max(),'Melhor (S/N)'] = 'S'

In [33]:
df_resultados.to_csv('resultados.csv', index=False)

# Lab da Semana

In [ ]:
b2wCorpus["review_text"]


## Pré-processamento
# <font color='blue'>Questão 1 </font>
Copie suas etapas de préprocessamento da lista 03, ou seja, selecione apenas as colunas relevantes ("review_text" e "recommend_to_a_friend"), converta a coluna "review_text" de uma coluna de `str` para uma coluna de `int` e separe os dados em teste e treino.


In [ ]:
# Seu código aqui

## Tokenizando




# <font color='blue'>Questão 2 </font>
Utilizando a camada [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) tokenize os inputs.
Declare a camada e então chame a função `adapt()` no seu conjunto de treino para adequar o seu vocabulário aos reviews.

Note que o uso de padding não é mais necessario.

In [ ]:
# Seu código aqui

## LSTM&GRU

Agora vamos juntar a camada do tokenizador a nossa camada [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) e definir o resto de nosso modelo.

#  <font color='blue'>Questão 3 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas  [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.



b) Como foi a performance desta rede em comparação a da lista 3?




**<font color='red'> Sua resposta aqui </font>**

In [ ]:
# Seu código aqui

#  <font color='blue'>Questão 4 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.



b) Como foi a performance desta rede em comparação a da lista 3?


**<font color='red'> Sua resposta aqui </font>**

In [ ]:
# Seu código aqui

## Redes Bi-direcionais
#  <font color='blue'>Questão 5 </font>

a) Defina, compile, treine e avalie um novo modelo que utilize contexto em ambas as direções usando a camada [`Bidirectional()`](https://keras.io/api/layers/recurrent_layers/bidirectional/), seja com camadas GRU ou LSTM.


b) Como foi sua performance em relação as questões anteriores com contexto unidirecional?

**<font color='red'> Sua resposta aqui </font>**

In [ ]:
# Seu código aqui